Test Wikibaseintegrator

Imports

In [22]:
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator.wbi_config import config

Config

In [23]:
config['USER_AGENT'] = 'test'

Create WikibaseIntegrator object


In [24]:
wbi = WikibaseIntegrator()

Get a single item


In [25]:
castle = wbi.item.get('Q1049154')

Get english label

In [26]:
castle.labels.get('en').value

'Castle of Carlos V, Hondarribia'

Get Country

In [49]:
country_id=castle.claims.get('P17')[0].mainsnak.datavalue['value']['id'] # get the country id
country=wbi.item.get(country_id) # get the country item
country.labels.get('fr').value # get the country label in french

Q29


'Espagne'